In [1]:
import pandas as pd
import os
def data_dictionary():
    path_train="dataset/"
    list_train=os.listdir(path_train)
    train_dictionary={"image_path":[],"target":[]}
    k=0
    for i in list_train:
        
        path_disease_train=path_train+i
        image_list_train=os.listdir(path_disease_train)
        for j in image_list_train:
            img_path_train=path_disease_train+"/"+j   
            train_dictionary["image_path"].append(img_path_train)
            train_dictionary['target'].append(k)     
        k+=1
    train_df=pd.DataFrame(train_dictionary)
    return  train_df

In [2]:
import gc
gc.collect()

0

In [3]:
data=data_dictionary()

In [4]:
lable=data['target']

In [5]:
import cv2
images=[]
label=[]
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
for i in data['image_path']:
    img = load_img(i, target_size=(100, 100))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    images.append(x) 

2023-07-21 12:44:27.129595: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-21 12:44:31.171455: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-07-21 12:44:31.172699: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PA

In [6]:
import numpy as np
import pandas as pd
dataset=np.array(images)


In [7]:
dataset = np.reshape(dataset, (23184, 100, 100, 3))

In [8]:
lable=np.array(lable)

In [9]:
import tensorflow as tf

In [ ]:
EFFModel=tf.keras.applications.EfficientNetB7( weights = 'imagenet',  include_top = False, input_shape = (100,100 ,3))

2023-07-21 12:55:29.712744: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-07-21 12:55:29.712786: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-21 12:55:29.712815: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-11-20230720-122152): /proc/driver/nvidia/version does not exist
2023-07-21 12:55:29.713994: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  A

In [13]:
EFFModel.summary()

Model: "efficientnetb7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 120, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 120, 120, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 120, 120, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 120, 120, 3)  0           ['normalization[0][0

In [14]:
# let's make all layers non-trainable
for layer in EFFModel.layers : 
    layer.trainable = False

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset, lable, test_size=0.2,random_state=44)
#X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2,random_state=1)


In [15]:
X_train.shape

(18547, 100, 100, 3)

In [16]:
del dataset

In [19]:
del images

In [20]:
import gc
gc.collect()


472

In [ ]:
features_train = EFFModel.predict(X_train)  


2023-07-21 15:01:35.454925: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2225640000 exceeds 10% of free system memory.


129/580 [=====>........................] - ETA: 6:29

In [23]:
features_train.shape

(18547, 4, 4, 2560)

In [24]:
features_test = EFFModel.predict(X_test) 

145/145 [==============================] - 121s 837ms/step


In [25]:
# Save the object to a file using pickle
import pickle
with open('features_train.pickle', 'wb') as f:
    pickle.dump(features_train, f)

In [26]:
# Save the object to a file using pickle
import pickle
with open('features_test.pickle', 'wb') as f:
    pickle.dump(features_test, f)

In [27]:
print(25)

25


In [28]:
num_train=features_train.shape[0]
num_test=features_test.shape[0]

In [29]:
x_test=features_test.reshape(num_test,-1)

In [30]:
x_train=features_train.reshape(num_train,-1)

In [31]:
del features_train

In [32]:
del features_test 

In [33]:
x_train.shape

(18547, 40960)

In [34]:
x_test.shape

(4637, 40960)

In [35]:
gc.collect()

2547

In [36]:
from sklearn.decomposition import PCA
pca = PCA(n_components =500) 
X_train = pca.fit_transform(x_train)
X_test = pca.transform(x_test)

In [37]:
pip install lazypredict


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.1 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 5.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [38]:
y_train.shape

(18547,)

In [39]:
from lazypredict.Supervised import LazyClassifier
clf= LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train,X_test,y_train,y_test)


 97%|█████████▋| 28/29 [54:38<04:06, 246.70s/it] 

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.043815 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 127500
[LightGBM] [Info] Number of data points in the train set: 18547, number of used features: 500
[LightGBM] [Info] Start training from score -3.140955
[LightGBM] [Info] Start training from score -3.132264
[LightGBM] [Info] Start training from score -3.126103
[LightGBM] [Info] Start training from score -3.093472
[LightGBM] [Info] Start training from score -3.143452
[LightGBM] [Info] Start training from score -3.153502
[LightGBM] [Info] Start training from score -3.161107
[LightGBM] [Info] Start training from score -3.133501
[LightGBM] [Info] Start training from score -3.143452
[LightGBM] [Info] Start training from score -3.128563
[LightGBM] [Info] Start training from score -3.161107
[LightGBM] [Info] Start training from score -3.111469
[LightGBM] [Info] Start training from score -3.147209
[Li

100%|██████████| 29/29 [55:53<00:00, 115.64s/it]


In [ ]:
predictions

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
QuadraticDiscriminantAnalysis,0.92,0.92,None,0.92,3.35
ExtraTreesClassifier,0.90,0.90,None,0.90,15.15
RandomForestClassifier,0.89,0.89,None,0.89,81.52
XGBClassifier,0.88,0.88,None,0.88,868.31
LGBMClassifier,0.88,0.88,None,0.88,75.41
SVC,0.84,0.84,None,0.84,159.57
KNeighborsClassifier,0.81,0.81,None,0.81,1.32
NuSVC,0.81,0.81,None,0.81,187.92
BaggingClassifier,0.72,0.72,None,0.72,206.23


In [52]:
import seaborn as sns 

# Create a count plot of the 'Gender' column in the dataframe
sns.countplot(x='Gender', data=predictions)

plt.show()

ValueError: Could not interpret input 'Gender'

In [49]:
model_name = models.index[0]
predictions[model_name]


KeyError: 'QuadraticDiscriminantAnalysis'